In [16]:
import pandas as pd

# Load the dataset
file_path = 'Weather Training Data.csv'
# Loading the dataset again to work with the original 'Location' values (string names)
weather_data = pd.read_csv(file_path)



In [17]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
import numpy as np

# Preprocessing steps for the whole dataset

# # Dropping 'row ID' as it's not needed for model training
# weather_data = weather_data.drop(['row ID'], axis=1)

# Handling missing values for numerical columns
num_cols = weather_data.select_dtypes(include=[np.number]).columns
imputer_num = SimpleImputer(strategy='median')
weather_data[num_cols] = imputer_num.fit_transform(weather_data[num_cols])

# Handling missing values for categorical columns
# For now, we will drop columns with a high percentage of missing values
weather_data = weather_data.drop(['Evaporation', 'Sunshine', 'Cloud9am', 'Cloud3pm'], axis=1)
cat_cols = weather_data.select_dtypes(include=['object']).columns
imputer_cat = SimpleImputer(strategy='most_frequent')
weather_data[cat_cols] = imputer_cat.fit_transform(weather_data[cat_cols])



# Encoding categorical variables
label_encoders = {}
for column in cat_cols:
    label_encoder = LabelEncoder()
    weather_data[column] = label_encoder.fit_transform(weather_data[column].astype(str))
    label_encoders[column] = label_encoder

# Adjusting the provided code to extract data for all coastal cities in Australia

# First, let's define the coastal cities as per the original dataset
coastal_cities = ['Adelaide', 'Albany', 'Brisbane', 'Cairns', 'CoffsHarbour', 
                  'Darwin', 'GoldCoast', 'Hobart', 'Melbourne', 'MelbourneAirport',
                  'Newcastle', 'NorahHead', 'NorfolkIsland', 'Perth', 'PerthAirport',
                  'Portland', 'Sydney', 'SydneyAirport', 'Townsville', 'Williamtown', 'Wollongong']

# Now, let's extract the encoded values for these coastal cities
encoded_coastal_cities = {label_encoders['Location'].transform([city])[0] for city in coastal_cities if city in label_encoders['Location'].classes_}

# Extracting data for these coastal cities
coastal_data = weather_data[weather_data['Location'].isin(encoded_coastal_cities)]

# Overview of the processed coastal cities data
coastal_data.head()




,row ID,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
6273,59784,11,16.1,31.4,0.0,5,54.0,3,5,7.0,37.0,51.0,58.0,1005.9,1002.3,26.5,28.4,0,0.0
6274,59795,11,22.8,24.7,0.0,11,56.0,11,8,35.0,15.0,68.0,67.0,1010.9,1011.4,23.4,24.4,0,1.0
6275,59805,11,14.8,25.0,0.8,0,24.0,10,2,7.0,17.0,62.0,45.0,1019.5,1017.0,22.5,24.8,0,0.0
6276,59816,11,15.5,27.3,0.0,4,41.0,7,4,7.0,30.0,54.0,62.0,1015.7,1012.7,24.6,26.1,0,0.0
6277,59827,11,22.7,29.2,0.0,11,44.0,7,5,9.0,20.0,70.0,71.0,1009.4,1008.8,25.8,26.4,0,1.0


In [14]:
# Mapping the encoded integer values back to their original string labels in 'Location' column
location_mappings = {index: label for index, label in enumerate(label_encoders['Location'].classes_)}
# location_mappings


In [18]:
from sklearn.model_selection import train_test_split

# Splitting the dataset into features (X) and target variable (y)
X = coastal_data.drop(columns=['RainTomorrow'])
y = coastal_data['RainTomorrow']

# Splitting the data into training, validation, and test sets
# Using 70% of data for training, 15% for validation, and 15% for testing
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

y_train = y_train.astype('int')
y_val = y_val.astype('int')
y_test = y_test.astype('int')

# Sizes of each dataset
sizes = {
    "Training set size": X_train.shape[0],
    "Validation set size": X_val.shape[0],
    "Test set size": X_test.shape[0]
}

sizes


{'Training set size': 30885,
 'Validation set size': 6618,
 'Test set size': 6619}

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV

# Initialize Logistic Regression model
logreg = LogisticRegression(solver='liblinear', random_state=42)

# Parameters for tuning
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100], # Regularization parameter
    'penalty': ['l1', 'l2'] # Norm used in the penalization
}

# Grid search with cross-validation for parameter tuning
grid_search = GridSearchCV(logreg, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_val, y_val)

# Best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Train the model with the best parameters on the full training set
logreg_optimized = LogisticRegression(**best_params, solver='liblinear', random_state=42)
logreg_optimized.fit(X_train, y_train)

# Predict on the test set
y_pred = logreg_optimized.predict(X_test)

# Evaluate the optimized model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Printing the best parameters found by Grid Search
print("Best Parameters from Grid Search:", best_params)

# Printing the best score achieved on the validation set during Grid Search
print("Best Score from Grid Search on Validation Set:", best_score)

# Printing the accuracy of the optimized model on the test set
print("Accuracy of Optimized Model on Test Set:", accuracy)

# Printing the detailed classification report for the test set predictions
print("Classification Report for Test Set:")
print(classification_rep)



Best Parameters from Grid Search: {'C': 0.1, 'penalty': 'l1'}
Best Score from Grid Search on Validation Set: 0.8136900480232375
Accuracy of Optimized Model on Test Set: 0.8116029611723825
Classification Report for Test Set:
              precision    recall  f1-score   support

           0       0.83      0.94      0.88      4860
           1       0.72      0.47      0.57      1759

    accuracy                           0.81      6619
   macro avg       0.78      0.70      0.72      6619
weighted avg       0.80      0.81      0.80      6619



In [5]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV

# Initialize the Support Vector Machine classifier
svm_classifier = SVC(random_state=42)

# Parameters for tuning
param_grid_svm = {
    'C': [0.1, 1, 10],  # Regularization parameter
    # 'kernel': ['rbf', 'poly'],  # Kernel types
    'kernel': ['rbf'],  # Kernel types
    'gamma': ['scale', 'auto']  # Kernel coefficient
}

# Grid search with cross-validation for parameter tuning
grid_search_svm = GridSearchCV(svm_classifier, param_grid_svm, cv=5, scoring='accuracy')
grid_search_svm.fit(X_val, y_val)

# Best parameters and best score
best_params_svm = grid_search_svm.best_params_
best_score_svm = grid_search_svm.best_score_

# Train the model with the best parameters on the full training set
svm_optimized = SVC(**best_params_svm, random_state=42)
svm_optimized.fit(X_train, y_train)

# Predict on the test set
y_pred_svm = svm_optimized.predict(X_test)

# Evaluate the optimized model
accuracy_svm = accuracy_score(y_test, y_pred_svm)
classification_rep_svm = classification_report(y_test, y_pred_svm)

# Printing the best parameters found by Grid Search
print("Best Parameters from Grid Search for SVM:", best_params_svm)

# Printing the best score achieved on the validation set during Grid Search
print("Best Score from Grid Search on Validation Set for SVM:", best_score_svm)

# Printing the accuracy of the optimized SVM model on the test set
print("Accuracy of Optimized SVM Model on Test Set:", accuracy_svm)

# Printing the detailed classification report for the test set predictions by the SVM model
print("Classification Report for Test Set using SVM:")
print(classification_rep_svm)



Best Parameters from Grid Search for SVM: {'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}
Best Score from Grid Search on Validation Set for SVM: 0.7763113850116848
Accuracy of Optimized SVM Model on Test Set: 0.769895498392283
Classification Report for Test Set using SVM:
              precision    recall  f1-score   support

           0       0.77      1.00      0.87     11493
           1       0.00      0.00      0.00      3435

    accuracy                           0.77     14928
   macro avg       0.38      0.50      0.43     14928
weighted avg       0.59      0.77      0.67     14928



/Users/xinranli/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/xinranli/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/xinranli/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
